In [2]:
%reload_ext autoreload
%matplotlib inline

import logging
import warnings
from pathlib import Path 

import math
import random
import numpy as np 
import pandas as pd 
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error

warnings.filterwarnings('ignore')
logging.basicConfig(level=logging.INFO, format='%(asctime)-15s %(levelname)s: %(message)s')

# ------------- Global Variable -------------
data_root = Path(r'data')
SEED =2021

In [3]:
def seed_everything(seed=2021):
    random.seed(seed)
    np.random.seed(seed)
    # torch.manual_seed(seed)
    # torch.cuda.manual_seed(seed)
    # torch.backends.cudnn.deterministic = True

def calc_mape(y_true, y_pred):
    ''' 平均绝对百分比误差 '''
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true-y_pred)/y_true)) * 100

seed_everything(SEED)

In [4]:
# Prepare Data

df = pd.read_csv(f'{data_root}/ak_usd-cny.csv')
df['Date'] = pd.to_datetime(df['Date'], format=r'%Y-%m-%d')
df.columns = [str(x).lower().replace(' ', '_') for x in df.columns] # Change all column headings to be lower case, and remove spacing
df.sort_values(by='date', inplace=True)
logging.info(f"data shape: {df.shape}")
df.head()

2021-02-02 22:26:06,478 INFO: data shape: (1414, 6)


,date,c,o,h,l,p
1413,2015-09-01,6.3645,6.3775,6.3775,6.3635,-0.0019
1412,2015-09-02,6.3559,6.3656,6.3668,6.3555,-0.0014
1411,2015-09-03,6.3559,6.3559,6.3559,6.3559,0.0000
1410,2015-09-04,6.3559,6.3559,6.3559,6.3559,0.0000
1409,2015-09-07,6.3659,6.3574,6.3704,6.3560,0.0016


In [5]:
# 需要分析的时间序列
ts_series = df.c

In [6]:
def arima_regression(ts, order=(1,1,1)):
    p, d, q = order

    training_data, test_data = ts[0: int(len(ts)*0.7)], ts[int(len(ts)*0.7):]

    history = [x for x in training_data]
    model_predictions = []

    # To this end, we re-create the ARIMA model after each new observation is received.
    for time_point in range(len(test_data)):
        model = ARIMA(history[-500:], order=(p, d, q))
        model_fit = model.fit(disp=0)
        output = model_fit.forecast()
        yhat = output[0]
        model_predictions.append(yhat)
        true_test_value = test_data[time_point]
        history.append(true_test_value)

    mse = mean_squared_error(test_data, model_predictions)
    return mse

模型调参

In [7]:
# 设置调参所需全局参数
ds = ts_series  # 数据集

def optim_arima(p_value, d_value, q_value):
    order = (int(round(p_value)), int(round(d_value)), int(round(q_value)))
    try:
        mse = arima_regression(ds, order)
    except:
        mse = 1000
    return np.exp(-mse)  # MSE越小越好, e^(-mse)越大越好

In [ ]:
%%time 
from bayes_opt import BayesianOptimization

model_params = {
    'p_value': (1, 10),
    'd_value': (1, 2),
    'q_value': (1, 10),
}

bayes_optim_arima = BayesianOptimization(optim_arima, model_params)
bayes_optim_arima.maximize(n_iter=30)

|   iter    |  target   |  d_value  |  p_value  |  q_value  |
-------------------------------------------------------------


2021-02-02 22:26:26,193 INFO: NumExpr defaulting to 2 threads.


|  1        |  0.0      |  1.477    |  2.888    |  6.283    |
|  2        |  0.9145   |  1.427    |  1.395    |  4.583    |
|  3        |  0.0      |  1.194    |  4.731    |  7.944    |
|  4        |  0.0      |  1.562    |  8.142    |  9.347    |
|  5        |  0.0      |  1.79     |  3.003    |  6.989    |
|  6        |  0.9145   |  1.405    |  1.0      |  3.535    |
|  7        |  0.0      |  2.0      |  2.946    |  2.306    |
|  8        |  0.9143   |  2.0      |  1.0      |  4.344    |
|  9        |  0.9145   |  1.0      |  10.0     |  1.0      |
|  10       |  0.0      |  2.0      |  10.0     |  3.329    |
|  11       |  0.9145   |  1.0      |  7.955    |  1.0      |
|  12       |  0.0      |  2.0      |  1.0      |  10.0     |
|  13       |  0.9143   |  1.971    |  8.928    |  1.019    |
|  14       |  0.9143   |  2.0      |  6.55     |  1.0      |
|  15       |  0.0      |  1.0      |  6.649    |  2.742    |
|  16       |  0.9146   |  1.0      |  1.0      |  1.0      |
|  17   

In [ ]:
bayes_optim_lstm.max

基于最优参数的模型测试